In [1]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n28", 5555)

connected to arkouda server tcp://*:5555


In [3]:
n = 100
m = 1000
s = 2
x = 2
y = 2

In [4]:
import time

### Build graph from randomly generated source and destination arrays.
# 1. Use Arkouda's randint to generate the random edge arrays.
src = ak.randint(0, n, m, seed=s*2)
dst = ak.randint(0, n, m, seed=s*4)

# 2. Build temporary property graph to get sorted edges and nodes lists.
temp_prop_graph = ar.PropGraph()
start = time.time()
temp_prop_graph.add_edges_from(src, dst)
end = time.time()
build_time = end - start
print(f"Building property graph with {len(temp_prop_graph)} vertices and "
        f"{temp_prop_graph.size()} "
        f"edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = temp_prop_graph.size()
num_nodes = len(temp_prop_graph)
edges = temp_prop_graph.edges()
nodes = temp_prop_graph.nodes()

# 2. Generate random integer attributes.
node_labels_1 = ak.randint(1, x+1, num_nodes)
node_labels_2 = ak.randint(1, x+1, num_nodes)
edge_rels_1 = ak.randint(1, y+1, num_edges)
edge_rels_2 = ak.randint(1, y+1, num_edges)

# 3. Create dataframe to load into a new property graph.
edge_df = ak.DataFrame({"src":edges[0], "dst":edges[1], "rels1":edge_rels_1, "rels2":edge_rels_2})
node_df = ak.DataFrame({"nodes":nodes, "lbls1":node_labels_1, "lbls2":node_labels_2})

# 4. Create new property graph with node labels and edge relationships.
prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst",
                                relationship_columns=["rels1", "rels2"])
prop_graph.load_node_attributes(node_df, node_column="nodes", label_columns=["lbls1", "lbls2"])

### Create the subgraph we are searching for.
# 1. Create labels and relationships to search for.
src_list = [0, 1, 2, 2, 3]
dst_list = [1, 2, 0, 3, 0]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

labels1_subgraph = ak.array([1, 1, 1, 1])
labels2_subgraph = ak.array([2, 2, 2, 2])
rels1_subgraph = ak.array([1, 1, 1, 1, 1])
rels2_subgraph = ak.array([2, 2, 2, 2, 2])

Building property graph with 100 vertices and 949 edges took 3.51 seconds.


In [5]:
"""TEST VERTEX PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), "lbls1":labels1_subgraph,
                          "lbls2":labels2_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column = "nodes", 
                                label_columns = ["lbls1","lbls2"])

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 218.0 isos inside of the graph


In [6]:
"""TEST EDGE PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                        "rels1":rels1_subgraph, "rels2":rels2_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst",
                                relationship_columns=["rels1","rels2"])

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 169.0 isos inside of the graph


In [7]:
"""TEST COMBINED PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                        "rels1":rels1_subgraph, "rels2":rels2_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), "lbls1":labels1_subgraph,
                          "lbls2":labels2_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst",
                                relationship_columns=["rels1","rels2"])
subgraph.load_node_attributes(node_df_h, node_column = "nodes", 
                                label_columns=["lbls1","lbls2"])

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 71.0 isos inside of the graph


In [8]:
"""TEST NO PICKER"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="si", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 670.0 isos inside of the graph


In [9]:
"""VF2-PS VERTEX PICKER GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), "lbls1":labels1_subgraph,
                          "lbls2":labels2_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column = "nodes", 
                                label_columns = ["lbls1","lbls2"])

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 218.0 isos inside of the graph


In [10]:
"""VF2-PS EDGE PICKER GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                        "rels1":rels1_subgraph, "rels2":rels2_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst",
                                relationship_columns=["rels1","rels2"])

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 169.0 isos inside of the graph


In [11]:
"""VF2-PS COMBINED GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                        "rels1":rels1_subgraph, "rels2":rels2_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.array(subgraph_nodes), "lbls1":labels1_subgraph,
                          "lbls2":labels2_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst",
                                relationship_columns=["rels1","rels2"])
subgraph.load_node_attributes(node_df_h, node_column = "nodes", 
                                label_columns=["lbls1","lbls2"])

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 71.0 isos inside of the graph


In [12]:
"""VF2-PS NO ATTRIBUTES GROUND TRUTH"""
subgraph = ar.PropGraph()
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check="or", algorithm_type="ps", return_isos_as="vertices"
)
print(f"We found {len(isos_as_vertices[0])/len(subgraph)} isos inside of the graph")

We found 670.0 isos inside of the graph
